In [1]:
pip install scikit-learn pandas numpy matminer pymatgen torch stable-baselines3 gym networkx psutil pynvml tqdm matplotlib joblib


     ------------------------------------- 721.7/721.7 kB 14.8 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   --------------------------------------- 958.1/958.1 kB 22.5 MB/s eta 0:00:00
  Created wheel for gym: filename=gym-0.26.2-py3-none-any.whl size=827633 sha256=8c4b41ea6fcc507eb2bf584e5c36dd1cd6e67782a011df513bc53b40dbca898b
  Stored in directory: c:\users\48103\appdata\local\pip\cache\wheels\17\79\65\7afedc162d858b02708a3b8f7a6dd5b1000dcd5b0f894f7cc1
Successfully built gym
  Attempting uninstall: ruamel.yaml
    Found existing installation: ruamel.yaml 0.18.11
    Uninstalling ruamel.yaml-0.18.11:
      Successfully uninstalled ruamel.yaml-0.18.11
Note: you may need to restart the 

In [3]:
# 初始化项目结构 — 在 Jupyter Notebook 中运行这一整段

# 1. 创建目录
import os

dirs = ["data/raw", "data/processed", "models", "logs", "dash_app/data"]
for d in dirs:
    os.makedirs(d, exist_ok=True)
    print(f"Created folder: {d}")

# 2. 创建基础脚本文件
files = {
    "pipeline.py": r"""\
# pipeline.py — baseline pipeline 模块
import os, time
import numpy as np
import pandas as pd
import joblib
import psutil
from mp_api.client import MPRester
from matminer.featurizers.composition import ElementProperty
from matminer.featurizers.structure import DensityFeatures, GlobalSymmetryFeatures
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_absolute_error

def run_pipeline(config, df):
    # TODO: 在此实现 N1–N6 步骤
    mae_val = 0.0
    total_cost = 0.0
    return mae_val, total_cost, None
""",
    "env.py": r"""\
# env.py — PPO 环境封装
import gym
import numpy as np
import networkx as nx
from pipeline import run_pipeline

class PipelineEnv(gym.Env):
    def __init__(self, df_train):
        super().__init__()
        self.df = df_train
        self.budget = 100.0
        self.config = {}

    def reset(self):
        self.config = {}  # TODO: 填充默认 config
        self.budget = 100.0
        mae, cost, _ = run_pipeline(self.config, self.df)
        return None  # TODO: 返回 state
        
    def step(self, action):
        # TODO: 应用 action 修改 config 并运行 pipeline
        return None, 0.0, False, {}

    def graph_consistent(self, config):
        # TODO: 实现 DAG 检查逻辑
        return True
""",
    "train_ppo.py": r"""\
# train_ppo.py — 训练入口脚本
from stable_baselines3 import PPO
from env import PipelineEnv
import pandas as pd

if __name__ == "__main__":
    df_train = pd.read_csv("data/processed/non_fe.csv")  # TODO: 修改为实际路径
    env = PipelineEnv(df_train)
    model = PPO("MlpPolicy", env, verbose=1, clip_range=0.2, target_kl=0.01)
    model.learn(total_timesteps=10000)
    model.save("models/ppo_pipeline.zip")
    print("PPO model trained and saved.")
"""
}

# 写入文件
for fname, content in files.items():
    with open(fname, "w", encoding="utf-8") as f:
        f.write(content)
    print(f"Created file: {fname}")


Created folder: data/raw
Created folder: data/processed
Created folder: models
Created folder: logs
Created folder: dash_app/data
Created file: pipeline.py
Created file: env.py
Created file: train_ppo.py


In [8]:
# env.py
import gym
from gym import spaces
import numpy as np
import networkx as nx
from pipeline import run_pipeline  # stub 版

class PipelineEnv(gym.Env):
    def __init__(self, df_train):
        super().__init__()
        self.df = df_train
        # 假设 7 个节点，每个最多 3 种方法
        self.n_nodes = 7
        self.n_methods = 3
        # action：node（7），method（3），两个 hyperparams 连续
        self.action_space = spaces.Dict({
            "node": spaces.Discrete(self.n_nodes),
            "method": spaces.Discrete(self.n_methods),
            "hp": spaces.Box(low=0.0, high=1.0, shape=(2,), dtype=np.float32)
        })
        # state：one-hot 方法选择 + mae + cost + 剩余预算
        self.observation_space = spaces.Box(low=0, high=1, shape=(self.n_nodes*self.n_methods + 3,), dtype=np.float32)
        self.budget = 100.0
        self.config = {}

    def reset(self, *, seed=None, options=None):
        super().reset(seed=seed)
        self.budget = 100.0
        self.config = {i: 0 for i in range(self.n_nodes)}  # 默认方法 0
        mae, cost, _ = run_pipeline(self.config, self.df)
        return self._build_state(mae, cost), {}

    def step(self, action):
        prev_config = self.config.copy()
        node = action["node"]
        method = action["method"]
        hp = action["hp"]
        self.config[node] = method
        if not self.graph_consistent():
            self.config = prev_config
            return self._build_state(0,0), -1.0, False, {}
        mae, cost, _ = run_pipeline(self.config, self.df)
        self.budget -= cost
        reward = -mae - 0.01 * cost  # example
        done = self.budget <= 0
        return self._build_state(mae, cost), reward, done, {}

    def _build_state(self, mae, cost):
        onehot = np.zeros(self.n_nodes*self.n_methods, dtype=np.float32)
        for n, m in self.config.items():
            onehot[n*self.n_methods + m] = 1.0
        return np.concatenate([onehot, [mae, cost, self.budget]])

    def graph_consistent(self):
        # 简单检查无环境断链；demo 中直接 return True
        return True


In [17]:
from env import PipelineEnv
import pandas as pd
df = pd.DataFrame()  # stub
env = PipelineEnv(df)
from stable_baselines3.common.env_checker import check_env
check_env(env)
from stable_baselines3.common.env_checker import check_env
env = PipelineEnv(pd.DataFrame())
check_env(env)


[autoreload of env failed: Traceback (most recent call last):
  File "D:\Anaconda\lib\site-packages\IPython\extensions\autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "D:\Anaconda\lib\site-packages\IPython\extensions\autoreload.py", line 410, in superreload
    update_generic(old_obj, new_obj)
  File "D:\Anaconda\lib\site-packages\IPython\extensions\autoreload.py", line 347, in update_generic
    update(a, b)
  File "D:\Anaconda\lib\site-packages\IPython\extensions\autoreload.py", line 302, in update_class
    if update_generic(old_obj, new_obj): continue
  File "D:\Anaconda\lib\site-packages\IPython\extensions\autoreload.py", line 347, in update_generic
    update(a, b)
  File "D:\Anaconda\lib\site-packages\IPython\extensions\autoreload.py", line 266, in update_function
    setattr(old, name, getattr(new, name))
ValueError: reset() requires a code object with 0 free vars, not 1
]
D:\Anaconda\lib\site-packages\stable_baselines3\common\env_checker.

In [12]:
pip install stable-baselines3==1.7.0 gym==0.21


     ---------------------------------------- 1.5/1.5 MB 16.4 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  python setup.py egg_info did not run successfully.
  exit code: 1
  
  [3 lines of output]
  D:\Anaconda\lib\site-packages\setuptools\_distutils\dist.py:261: UserWarning: Unknown distribution option: 'tests_require'
    warnings.warn(msg)
  error in gym setup command: 'extras_require' must be a dictionary whose values are strings or lists of strings containing valid project/version requirement specifiers.
  [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: metadata-generation-failed

Encountered error while generating package metadata.

See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
